# 1. Crawler

### 1.0.1. How to parse a page?

If you build a crawler, you might follow one of the approaches:
1. search for URLs in the page, assuming this is just a text.
2. search for URLs in the places where URLs should appear: `<a href=..`, `<img src=...`, `<iframe src=...` and so on.

To follow the first approach you can rely on some good regular expression. [Like this](https://stackoverflow.com/a/3809435).

To follow the second approach just read one of these: [short answer](https://stackoverflow.com/questions/1080411/retrieve-links-from-web-page-using-python-and-beautifulsoup) or [exhaustive explanation](https://hackersandslackers.com/scraping-urls-with-beautifulsoup/).

## 1.1. [15] Download and persist #
Please complete a code for `load()`, `download()` and `persist()` methods of `Document` class. What they do:
- for a given URL `download()` method downloads binary data and stores in `self.content`. It returns `True` for success, else `False`.
- `persist()` method saves `self.content` somewhere in file system. We do it to avoid multiple downloads (for caching in other words).
- `load()` method loads data from hard drive. Returns `True` for success.

Tests checks that your code somehow works.

**NB Passing the test doesn't mean you correctly completed the task.** These are **criteria, which have to be fullfilled**:
1. URL is a unique identifier (as it is a subset of URI). Thus, documents with different URLs should be stored in different files. Typical errors: documents from the same domain are overwritten to the same file, URLs with similar endings are downloaded to the same file, etc.
2. The document can be not only a text file, but also a binary. Pay attention that if you download `mp3` file, it still can be played. Hint: don't hurry to convert everything to text.

In [1]:
import os
import hashlib
from requests import ConnectTimeout
import requests
from urllib.parse import quote

class Document:

    DIRECTORY = 'storage'

    def __init__(self, url: str):
        self.url = url
        self.content: bytes = None
        
    def get(self):
        if not self.load():
            if not self.download():
                raise FileNotFoundError(self.url)
            else:
                self.persist()
    
    def download(self):
        print(f"GET {self.url}")
        try:
            r = requests.get(self.url, timeout=3)
        except ConnectTimeout:
            return False

        if r.status_code != 200:
            print("too bad")
            return False
        self.content = r.content
        return True

    def _filename(self):
        return f'{self.DIRECTORY}/{hashlib.sha1(self.url.encode()).digest().hex()}'

    def persist(self):
        print(f"PERSIST {self.url}")
        os.makedirs(self.DIRECTORY, exist_ok=True)
        with open(self._filename(), 'wb') as f:
            f.write(self.content)

    def load(self):
        print(f"LOAD {self.url}")
        try:
            with open(self._filename(), 'rb') as f:
                self.content = f.read()
                return True
        except FileNotFoundError:
            return False

### 1.1.1. Tests ###

In [2]:
doc = Document('http://sprotasov.ru/data/iu.txt')

doc.get()
assert doc.content, "Document download failed"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document content error"

doc.get()
assert doc.load(), "Load should return true for saved document"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document load from disk error"

LOAD http://sprotasov.ru/data/iu.txt
LOAD http://sprotasov.ru/data/iu.txt
LOAD http://sprotasov.ru/data/iu.txt


## 1.2. [M][15] Account the caching policy

Sometimes remote documents (especially when we speak about static content like `js` or `gif`) can swear that they will not change for some time. This is done by setting [Cache-Control response header](https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Cache-Control).

In [3]:
import requests
requests.get('https://polyfill.io/v3/polyfill.min.js').headers['Cache-Control']

'public, s-maxage=31536000, max-age=604800, stale-while-revalidate=604800, stale-if-error=604800'

Please study the documentation and implement a descendant to a `Document` class, which will refresh the document in case of expired cache even if the file is already on the hard drive.

In [4]:
class CachedDocument(Document):

    def __init__(self, url):
        self.expires = 0
        super().__init__(url)

    def download(self):
        print(f"GET {self.url}")
        r = requests.get(self.url)
        if r.status_code != 200:
            print("too bad")
            return False
        self.content = r.content
        cache = [ x.strip().split('=') for x in r.headers['Cache-Control'].split(',') ]
        cache = { x[0] : True if len(x) < 2 else x[1] for x in cache }
        maxage = float(cache.get('max-age', 0))
        self.expires = time.time() + maxage
        return True

    def load(self):
        if time.time() > self.expires:
            print(f"STALE {self.url}")
            return False
        print(f"LOAD {self.url}")
        try:
            with open(self._filename(), 'rb') as f:
                self.content = f.read()
                return True
        except FileNotFoundError:
            return False

### 1.2.1. Tests

Add logging in your code and show that your code behaves differently for documents with different caching policy.

In [5]:
import time

doc = CachedDocument('https://polyfill.io/v3/polyfill.min.js')
doc.get()
time.sleep(2)
doc.get()
time.sleep(2)
doc.get()

doc = CachedDocument('https://yandex.ru/')
doc.get()
time.sleep(2)
doc.get()
time.sleep(2)
doc.get()

STALE https://polyfill.io/v3/polyfill.min.js
GET https://polyfill.io/v3/polyfill.min.js
PERSIST https://polyfill.io/v3/polyfill.min.js
LOAD https://polyfill.io/v3/polyfill.min.js
LOAD https://polyfill.io/v3/polyfill.min.js
STALE https://yandex.ru/
GET https://yandex.ru/
PERSIST https://yandex.ru/
STALE https://yandex.ru/
GET https://yandex.ru/
PERSIST https://yandex.ru/
STALE https://yandex.ru/
GET https://yandex.ru/
PERSIST https://yandex.ru/


## 1.3. [10] Parse HTML ##
`BeautifulSoap` library is a de facto standard to parse XML and HTML documents in python. Use it to complete `parse()` method that extracts document contents. You should initialize:
- `self.anchors` list of tuples `('text', 'url')` met in a document. Be aware, there exist relative links (e.g. `../content/pic.jpg`). Use `urllib.parse.urljoin()` to fix this issue.
- `self.images` list of images met in a document. Again, links can be relative to current page.
- `self.text` should keep plain text of the document without scripts, tags, comments and so on. You can refer to [this stackoverflow answer](https://stackoverflow.com/a/1983219) for details.

**NB All these 3 criteria must be fulfilled to get full point for the task.**

In [6]:
from typing import List
from bs4 import BeautifulSoup
from bs4.element import Comment, Tag
from urllib.parse import urljoin


class HtmlDocument(Document):

    def tag_visible(self, element: Tag):
        if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
            return False
        if isinstance(element, Comment):
            return False
        return True


    def text_from_html(self, bs: Tag):
        texts = bs.findAll(text=True)
        visible_texts = filter(self.tag_visible, texts)
        return " ".join(t.strip() for t in visible_texts)

    def parse(self):
        content = self.content.decode('utf-8')
        bs = BeautifulSoup(content)
        aa: List[Tag] = bs.find_all('a')
        imgimg: List[Tag] = bs.find_all('img')

        self.anchors = [ (x.text, urljoin(self.url, x.attrs.get('href'))) for x in aa ]
        self.anchors = [ (t, u) for (t, u) in self.anchors if u ]
        self.images = [ urljoin(self.url, img.attrs['src']) for img in imgimg ]

        body = bs.find('body')
        self.text = self.text_from_html(body) if body else ""

### 1.3.1. Tests ###

In [7]:
doc = HtmlDocument("http://sprotasov.ru")
doc.get()
doc.parse()

assert "just few links" in doc.text, "Error parsing text"
assert "http://sprotasov.ru/images/gb.svg" in doc.images, "Error parsing images"
assert any(p[1] == "https://twitter.com/07C3" for p in doc.anchors), "Error parsing links"

LOAD http://sprotasov.ru


## 1.4. [10] Document analysis ##
Complete the code for `HtmlDocumentTextData` class. Implement word and sentence splitting (use any method you can propose). 

**Criteria of success**: 
1. Your `get_word_stats()` method should return `Counter` object.
2. Don't forget to lowercase your words for counting.
3. Sentences should be obtained inside `<body>` tag only.

In [8]:
from collections import Counter
import re

class HtmlDocumentTextData:
    def __init__(self, url):
        self.doc = HtmlDocument(url)
        self.doc.get()
        self.doc.parse()
    
    def get_sentences(self):
        return [ x.strip() for x in re.split("[,\n\r\t]+", self.doc.text) if x.strip() ]

    def get_word_stats(self):
        ctr = Counter()
        for x in self.doc.text.split():
            x = x.strip(',.:/?[]()!\\| ')
            x = x.lower()
            ctr[x] += 1
        return ctr

### 1.4.1. Tests ###

In [24]:
doc = HtmlDocumentTextData("https://innopolis.university/")

print(doc.get_word_stats().most_common(10))
assert [x for x in doc.get_word_stats().most_common(10) if x[0] == 'иннополис'], 'иннополис should be among most common'

print(*doc.get_sentences(), sep = '\n')

LOAD https://innopolis.university/
[('и', 67), ('в', 40), ('иннополис', 22), ('по', 13), ('для', 12), ('центр', 12), ('университета', 11), ('на', 11), ('с', 10), ('лаборатория', 10)]
Все медиа  Facebook Вконтакте Youtube Twitter Instagram habr      Абитуриентам  Бизнесу  Медиа   Личный кабинет      Университет      Об университете     Органы управления     Учредители    Наблюдательный совет      Команда университета     Организационная структура    Образовательные и научные подразделения    Технологические центры      Преподавательский состав     Профессорско-преподавательский состав    Вакантные должности ППС      Работа в университете     Карьера в университете    Корпоративная жизнь    Релокация в Иннополис    Вакансии      Кампус     Кампус  Информация о жилом
учебном и спортивном комплексах
медцентре
питании и досуге на территории города и Университета Иннополис. Ответы на часто задаваемые вопросы      Сведения об образовательной организации     Сведения об образовательной организ

## 1.5. [M][35] Languages ##

Maybe you heard, that there are multiple languages in the world. European languages, like Russian and English, use similar puctuation, but even in this family there is ¡Spanish!

Other languages can use different punctiation rules, like Arabic or Thai.

Your task is to support (at least) three languages (English, Arabic, and Thai) tokenization in your HtmlDocumentTextData class descendant.

What should you do:

1. Use any language dection techniques, e.g. langdetect.
2. Use language-specific tokenization tools, e.g. for Thai and Arabic.
3. Use these pages to test your code: 1 and 2.

In [20]:
import langdetect
from pythainlp import sent_tokenize as thai_tokenize
from camel_tools.tokenizers.word import simple_word_tokenize as arab_tokenize



class MultilingualHtmlDocumentTextData(HtmlDocumentTextData):

    def _lang(self):
        return langdetect.detect(self.doc.text)

    def get_word_stats(self):
        lang = self._lang()
        print(f"Lang is {lang}")
        if lang == 'th':
            tokens = thai_tokenize(self.doc.text, keep_whitespace=False)
        elif lang == 'ar':
            tokens = arab_tokenize(self.doc.text)
        else:
            raise NotImplementedError()

        ctr = Counter()
        for x in tokens:
            x = x.strip(',.:/?[]()!\\| ')
            x = x.lower()
            ctr[x] += 1
        return ctr

### 1.5.1. Tests ###

In [21]:
doc = MultilingualHtmlDocumentTextData("https://www.bangkokair.com/tha/baggage-allowance")
print(doc.get_word_stats().most_common(10))

doc = MultilingualHtmlDocumentTextData("https://alfajr-news.net/details/%D9%85%D8%B4%D8%B1%D9%88%D8%B9-%D8%AF%D9%8A%D9%85%D9%88%D9%82%D8%B1%D8%A7%D8%B7%D9%8A-%D9%81%D9%8A-%D8%A7%D9%84%D9%83%D9%88%D9%86%D8%BA%D8%B1%D8%B3-%D8%A7%D9%84%D8%A3%D9%85%D8%B1%D9%8A%D9%83%D9%8A-%D9%84%D9%85%D8%B9%D8%A7%D9%82")
print(doc.get_word_stats().most_common(10))


LOAD https://www.bangkokair.com/tha/baggage-allowance
Lang is th
[('ตรวจสอบรายละเอียด ที่นี่', 1), ('สำหรับข่าวประกาศต่างๆ เกี่ยวกับ covid-19 รวมถึงตารางปรับลดเที่ยวบินของบางกอกแอร์เวย์ส ×        เว็บไซต์นี้มีการใช้งานคุกกี้ (cookies) เพื่อจัดการข้อมูลส่วนบุคคลและช่วยเพิ่มประสิทธ์ภาพในการใช้งานเว็บไซต์ของท่าน', 1), ('ท่านสามารถศึกษารายละเอียดเพิ่มเติมและการตั้งค่าคุกกี้ได้ที่ นโยบายการใช้คุ้กกี้ โดย คลิกที่นี่               ภาษาไทย         english      ภาษาไทย       繁體中文      简体中文            สกุลเงิน : usd     thb  sgd  myr  usd  gbp  eur  cny  jpy       เข้าสู่ระบบ  สมัครสมาชิกฟลายเออร์โบนัส                            ภาษาไทย         english      ภาษาไทย       繁體中文      简体中文                       ประกาศระงับเที่ยวบินในประเทศทุกเส้นทาง   บางกอกแอร์เวย์สยกเลิกเที่ยวบินภายในประเทศทุกเส้นทางเป็นการชั่วคราว ระหว่างวันที่ 7-30 เมษายน 2563 ผู้โดยสารสามารถดำเนินการเพื่อขอรับเงินคืนผ่านทาง www.bangkokair.com/refund หรือติดต่อศูนย์บริการข้อมูลลูกค้า โทร 1771 (ตลอด 24 ชั่วโมง) 02 270 6699 หรือ ส

## 1.5. [15] Crawling ##

Method `crawl_generator()` is given starting url (`source`) and max depth of search. It should return a **generator** of `HtmlDocumentTextData` objects (return a document as soon as it is downloaded and parsed). You can benefit from `yield obj_name` python construction. Use `HtmlDocumentTextData.anchors` field to go deeper.

In [22]:
class Crawler:

    def __init__(self):
        self.seen = set()

    def crawl_generator(self, source, depth=1):
        self.seen.add(source)

        try:
            cur = HtmlDocumentTextData(source)
        except (FileNotFoundError, UnicodeDecodeError):
            return

        yield cur

        if depth >= 1:
            x: str
            for (_, x) in cur.doc.anchors:
                if not x in self.seen \
                        and x[-4:] not in ('.pdf', '.mp3', '.avi', '.mp4', '.txt')\
                        and not x.startswith('mailto:'):
                    for c in self.crawl_generator(x, depth - 1):
                        yield c

### 1.5. Tests ###

In [23]:
crawler = Crawler()
counter = Counter()

for c in crawler.crawl_generator("https://innopolis.university/en/", 2):
    print(c.doc.url)
    if c.doc.url[-4:] in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'):
        print("Skipping", c.doc.url)
        continue
    counter.update(c.get_word_stats())
    print(len(counter), "distinct word(s) so far")
    
print("Done")

print(counter.most_common(20))
assert [x for x in counter.most_common(20) if x[0] == 'innopolis'], 'innopolis sould be among most common'

LOAD https://innopolis.university/en/
https://innopolis.university/en/
346 distinct word(s) so far
LOAD https://apply.innopolis.university/en
https://apply.innopolis.university/en
1187 distinct word(s) so far
LOAD https://corporate.innopolis.university/en
https://corporate.innopolis.university/en
1346 distinct word(s) so far
LOAD https://media.innopolis.university/en
https://media.innopolis.university/en
1401 distinct word(s) so far
LOAD https://innopolis.university/lk/
https://innopolis.university/lk/
1761 distinct word(s) so far
LOAD https://innopolis.university/en/about/
https://innopolis.university/en/about/
1899 distinct word(s) so far
LOAD https://innopolis.university/en/board/
https://innopolis.university/en/board/
1988 distinct word(s) so far
LOAD https://innopolis.university/en/team/
https://innopolis.university/en/team/
1989 distinct word(s) so far
LOAD https://innopolis.university/en/team-structure/
https://innopolis.university/en/team-structure/
1993 distinct word(s) so far